In [1]:
import os, sys
import numpy as np
import json
import neuroglancer
import pandas as pd
import ast
from matplotlib import pyplot as plt
import cv2
viewer = neuroglancer.Viewer()
print(viewer)

http://127.0.0.1:37211/v/253feece9e34816d631c149da6d7ea195361a56f/


In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)

from utilities.atlas.utilities_contour import get_contours_from_annotations, add_structure_to_neuroglancer, \
    create_full_volume

Connecting dklab@192.168.1.12:3306


In [11]:
xy_ng_resolution_um = 5
color_radius = 3
animal = 'MD589'

CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
csvfile = os.path.join(CSV_PATH, f'{animal}_corrected_vertices.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))
hand_annotations['name'] = hand_annotations['structure']
#hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))

In [12]:
hand_annotations.head()

,structure,section,vertices,name
0,VCA,94,"[[1022.13439444,758.34760113],[1028.13669367,7...",VCA
1,VCA,95,"[[1076.00402716,790.21823078],[1067.00066368,7...",VCA
2,VCA,96,"[[1017.68250619,825.98166826],[1008.85746611,8...",VCA
3,VCA,97,"[[1092.0726917,804.5778103],[1085.97980541,800...",VCA
4,VCA,98,"[[1051.07068571,832.19302931],[1043.20849496,8...",VCA


In [5]:
original_sizes = {}
original_offsets = {}
PADDED_SIZE = [47000, 23300]
with open('MD594.csv', 'r') as file:
    for line in file.readlines():
        first, second = line.split()
        section = float(first[:-4])
        x, y = map(float, second.split('x'))
        original_sizes[section] = (x, y)

        original_offsets[section] = ((PADDED_SIZE[0] - x) / 4, (PADDED_SIZE[1] - y) / 4) 

FileNotFoundError: [Errno 2] No such file or directory: 'MD594.csv'

In [13]:
contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, 'SC', hand_annotations, densify=4)

KeyError: 'side'

In [ ]:
volume = []
for section in list(contour_annotations.keys())[:2]:
    vertices = np.array(contour_annotations[section]['SC'][1])
    vertices = (vertices * 460) / 452

    volume_slice = np.zeros(PADDED_SIZE, dtype=np.uint8)
    points = (vertices + np.array(original_offsets[section])).astype(np.int32)
    volume_slice = cv2.polylines(volume_slice, [points], True, 1, 10, lineType=cv2.LINE_AA)
    volume.append(volume_slice)
    
volume = np.array(volume).sum(axis=0)

In [ ]:
min_max = np.array(min_max)
print(min_max.max(axis=0))
min_max.min(axis=0)

In [ ]:
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=section_137.reshape(section_137.shape[0], section_137.shape[1], 1), 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[452, 452, 20000]), 
        voxel_offset=(0, 0, 137)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:
std_structures = get_structures()
structures_arr = hand_annotations.name.unique()
annotation_structures = structures_arr.tolist()
structures = [a for a in annotation_structures if a in std_structures]

In [ ]:
color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}

In [ ]:
#Litao, this creates the volumes and origins from the hand annotations. This is the part that
# needs to really be worked on
structure_volume_origin = {}

for structure in structures:
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure)
        try:
            color = colors[sided]
        except:
            color = 100

    print(structure, color, end="\t")
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations,
                                                                             densify=4)
    if first_sec == 0 or last_sec == 0:
        print('No sections found')
        continue
    else:
        print('Section start, end:', first_sec, last_sec)

    threshold = 1
    volume, xyz_offsets = create_full_volume(contour_annotations, structure, first_sec, last_sec, \
                                                       color_radius, xy_ng_resolution_um, threshold, color)
    volume = np.swapaxes(volume, 0, 2)
    structure_volume_origin[structure] = (volume, xyz_offsets)

In [ ]:
x_length = 6000
y_length = 3000
z_length = 300
full_brain_volume_annotated = np.zeros((x_length, y_length, z_length), dtype=np.uint8)

for structure, (volume, origin) in structure_volume_origin.items():  
    
    x, y, z = origin
    x_start = int(x) #+ x_length // 2
    y_start = int(y) #+ y_length // 2
    z_start = int(z) #+ z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + volume.shape[2]
    
    print(structure,'X range', x_start, x_end, end="\t")
    print('Y range', y_start, y_end, end="\t")
    print('Z range', z_start, z_end)

    print(volume.shape)
    print(origin)
    full_brain_volume_annotated[x_start:x_end, y_start:y_end,z_start:z_end] += volume

In [ ]:
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[5, 5, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:
color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}
    
surround = False
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f) and 'test' not in f])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f) and 'test' not in f])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break
    
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume = np.load(os.path.join(VOL_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(VOL_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())